In [926]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [927]:
# Import usual modules
import pandas as pd
import csv
import math
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import openpyxl
import datetime
from scipy.stats import lognorm
import re
import string
from bs4 import BeautifulSoup
import requests
import unicodedata # for removing accented characters
import datetime



In [1951]:
# Extract timed event records

import pandas_gbq
from google.oauth2 import service_account

credentials = service_account.Credentials.from_service_account_file(
    '/Users/veesheenyuen/Desktop/DataScience/Keys/saa-analytics-7c8937b70609.json',
    
    
)

sql1="""
SELECT NAME, RESULT, AGE, RANK AS COMPETITION_RANK, EVENT, DOB, COUNTRY, CATEGORY_EVENT, GENDER, COMPETITION, DATE
FROM `saa-analytics.results.saa_full` 
WHERE RESULT!='NM' AND RESULT!='-' AND RESULT!='FOUL' AND RANK!='DNS' AND RESULT!='DNS' AND RESULT!='DNF' AND RESULT!='DNQ' AND RESULT!='DQ' AND RESULT IS NOT NULL
"""

athletes = pandas_gbq.read_gbq(sql1, project_id="saa-analytics", credentials=credentials)




Downloading: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████|


In [1952]:
athletes

,NAME,RESULT,AGE,COMPETITION_RANK,EVENT,DOB,COUNTRY,CATEGORY_EVENT,GENDER,COMPETITION,DATE
0,REUBEN SEET HUIJIE,19:04.4,None,11.0,5000m,None,None,Long,Male,NSG,2023
1,PRANAV SHREEDHAR,18:58.1,None,5.0,5000m,None,None,Long,Male,NSG,2023
2,JACOB TAN,18:58.2,None,6.0,5000m,None,None,Long,Male,NSG,2023
3,SAMUEL TOH JUN XIAN,19:03.8,None,8.0,5000m,None,None,Long,Male,NSG,2023
4,LIEU YEA JAY ERWIN,19:04.1,None,9.0,5000m,None,None,Long,Male,NSG,2023
...,...,...,...,...,...,...,...,...,...,...,...
21486,"{435: 'Peh, Ying Xuan Ulyss', 324: 'Tan, Codee...",1:02.81,None,12,Girls 10-12 4x100 Meter Relay U13,None,None,Relay,Female,10th Club ZOOM Kindred Spirit Series 2023,2023
21487,"{799: 'Yu Xuan, Lua', 802: 'Annabelle Jiamin, ...",58.60,None,4,Girls 10-12 4x100 Meter Relay U13,None,None,Relay,Female,10th Club ZOOM Kindred Spirit Series 2023,2023
21488,"{983: 'Shah, Sharleez Eimann', 985: 'Pugsley, ...",1:09.86,None,18,Girls 10-12 4x100 Meter Relay U13,None,None,Relay,Female,10th Club ZOOM Kindred Spirit Series 2023,2023
21489,"{1309: 'Ong Chun Yi, Avelyn', 1308: 'Verena Ki...",1:03.22,None,14,Girls 10-12 4x100 Meter Relay U13,None,None,Relay,Female,10th Club ZOOM Kindred Spirit Series 2023,2023


In [1953]:
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+200 Meter Dash.+', value='200m')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+100 Meter Dash.+', value='100m')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+400 Meter Dash.+', value='400m')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+800 Meter Run.+', value='800m')

athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+1500 Meter Run.+', value='1500m run')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+3000 Meter Run.+', value='3000m run')

athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+110 Meter Hurdles.+', value='110m hurdles')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+100 Meter Hurdles.+', value='100m hurdles')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+400 Meter Hurdles.+', value='400m hurdles')


athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+4x100 Meter Relay.+', value='4 x 100m relay')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+4x400 Meter Relay.+', value='4 x 400m relay')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+3000 meter.+', value='3000m')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+5000 Meter Race Walk.+', value='5000m race walk')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+5000 Meter Run.+', value='5000m run')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+10000 Meter Run.+', value='10000m run')


athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+Race Walk.+', value='race walk')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+Pole Vault.+', value='Pole vault')

athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+Shot Put.+', value='Shot put')

athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+Discus.+', value='Discus throw')

mask = athletes['EVENT'].str.contains(r'Discus', na=True)
athletes.loc[mask, 'EVENT'] = 'Discus throw'

mask = athletes['EVENT'].str.contains(r'Shot', na=True)
athletes.loc[mask, 'EVENT'] = 'Shot put'

mask = athletes['EVENT'].str.contains(r'Javelin', na=True)
athletes.loc[mask, 'EVENT'] = 'Javelin throw'



athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+Triple Jump.+', value='Triple jump')

athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+Javelin Throw.+', value='Javelin throw')

athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+Long Jump.+', value='Long jump')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'Long Jump', value='Long jump')


athletes['EVENT'] = athletes['EVENT'].replace(regex=r'High Jump', value='High jump')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+High Jump.+', value='High jump')

athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+S/C.+', value='steeplechase')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+3000 Meter Steeplechase.+', value='3000m steeplechase')

mask = athletes['EVENT'].str.contains(r'High', na=True)
athletes.loc[mask, 'EVENT'] = 'High jump'

mask = athletes['EVENT'].str.contains(r'110m Hurdles', na=True)
athletes.loc[mask, 'EVENT'] = '110m hurdles'

mask = athletes['EVENT'].str.contains(r'400m Hurdles', na=True)
athletes.loc[mask, 'EVENT'] = '400m hurdles'

mask = athletes['EVENT'].str.contains(r'200m Hurdles', na=True)
athletes.loc[mask, 'EVENT'] = '200m hurdles'

mask = athletes['EVENT'].str.contains(r'100m Hurdles', na=True)
athletes.loc[mask, 'EVENT'] = '100m hurdles'

mask = athletes['EVENT'].str.contains(r'4 X 100m Relay', na=True)
athletes.loc[mask, 'EVENT'] = '4 x 100m relay'

mask = athletes['EVENT'].str.contains(r'4 X 400m Relay', na=True)
athletes.loc[mask, 'EVENT'] = '4 x 400m relay'

mask = athletes['EVENT'].str.contains(r'2000 Meter Steeplechase', na=True)
athletes.loc[mask, 'EVENT'] = '2000m steeplechase'

mask = athletes['EVENT'].str.contains(r'Hammer Throw', na=True)
athletes.loc[mask, 'EVENT'] = 'Hammer throw'

mask = athletes['EVENT'].str.contains(r'3000m S/C', na=True)
athletes.loc[mask, 'EVENT'] = '3000m steeplechase'

mask = athletes['EVENT'].str.contains(r'4x100m Relay', na=True)
athletes.loc[mask, 'EVENT'] = '4 x 100m relay'


# correct javelin category
mask = athletes['EVENT'].str.contains(r'Javelin', na=True)
athletes.loc[mask, 'CATEGORY_EVENT'] = 'Throw'




In [1954]:
athletes

,NAME,RESULT,AGE,COMPETITION_RANK,EVENT,DOB,COUNTRY,CATEGORY_EVENT,GENDER,COMPETITION,DATE
0,REUBEN SEET HUIJIE,19:04.4,None,11.0,5000m,None,None,Long,Male,NSG,2023
1,PRANAV SHREEDHAR,18:58.1,None,5.0,5000m,None,None,Long,Male,NSG,2023
2,JACOB TAN,18:58.2,None,6.0,5000m,None,None,Long,Male,NSG,2023
3,SAMUEL TOH JUN XIAN,19:03.8,None,8.0,5000m,None,None,Long,Male,NSG,2023
4,LIEU YEA JAY ERWIN,19:04.1,None,9.0,5000m,None,None,Long,Male,NSG,2023
...,...,...,...,...,...,...,...,...,...,...,...
21486,"{435: 'Peh, Ying Xuan Ulyss', 324: 'Tan, Codee...",1:02.81,None,12,4 x 100m relay,None,None,Relay,Female,10th Club ZOOM Kindred Spirit Series 2023,2023
21487,"{799: 'Yu Xuan, Lua', 802: 'Annabelle Jiamin, ...",58.60,None,4,4 x 100m relay,None,None,Relay,Female,10th Club ZOOM Kindred Spirit Series 2023,2023
21488,"{983: 'Shah, Sharleez Eimann', 985: 'Pugsley, ...",1:09.86,None,18,4 x 100m relay,None,None,Relay,Female,10th Club ZOOM Kindred Spirit Series 2023,2023
21489,"{1309: 'Ong Chun Yi, Avelyn', 1308: 'Verena Ki...",1:03.22,None,14,4 x 100m relay,None,None,Relay,Female,10th Club ZOOM Kindred Spirit Series 2023,2023


In [1955]:
athletes.to_csv('athletes.csv', sep=',', encoding='utf-8-sig', index=False)


In [1956]:
credentials = service_account.Credentials.from_service_account_file(
    '/Users/veesheenyuen/Desktop/DataScience/Keys/saa-analytics-7c8937b70609.json',
)

sql="""
SELECT NAME, RESULT, RANK, EVENT, CATEGORY_EVENT, GENDER, COMPETITION, STAGE
FROM `saa-analytics.results.saa_full`
WHERE STAGE='Final' AND COMPETITION='SEA Games' AND RANK='3'
"""

benchmarks = pandas_gbq.read_gbq(sql, project_id="saa-analytics", credentials=credentials)




Downloading: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████|


In [1957]:
benchmarks

,NAME,RESULT,RANK,EVENT,CATEGORY_EVENT,GENDER,COMPETITION,STAGE
0,Muhammad Haiqal Hanafi,10.443,3,100m,Sprint,Male,SEA Games,Final
1,Lalu Muhammad Zohri,21.02,3,200m,Sprint,Male,SEA Games,Final
2,Frederick Ramirez,46.63,3,400m,Sprint,Male,SEA Games,Final
3,Wan Muhammad Fazri Wan Zahari,1:53.86,3,800m,Mid,Male,SEA Games,Final
4,Robi Syianturi,14:43.45,3,5000m,Long,Male,SEA Games,Final
5,Tan Htike Soe,31:25.55,3,10000m,Sprint,Male,SEA Games,Final
6,John Cabang,13.855,3,110m hurdles,Hurdles,Male,SEA Games,Final
7,Calvin Quek,50.75,3,400m hurdles,Hurdles,Male,SEA Games,Final
8,Pandu Sukarya,8:55.05,3,3000m steeplechase,Steeple,Male,SEA Games,Final
9,"Jonathan Nyepa, Khairul Hafiz Jantan, Mohama...",39.36,3,4 x 100m relay,Relay,Male,SEA Games,Final


In [1958]:
benchmarks.rename(columns = {'RESULT':'BENCHMARK'}, inplace = True)


In [1959]:
benchmarks.drop(['NAME', 'RANK', 'CATEGORY_EVENT', 'COMPETITION', 'STAGE'], axis=1, inplace=True)


In [1960]:
benchmarks

,BENCHMARK,EVENT,GENDER
0,10.443,100m,Male
1,21.02,200m,Male
2,46.63,400m,Male
3,1:53.86,800m,Male
4,14:43.45,5000m,Male
5,31:25.55,10000m,Male
6,13.855,110m hurdles,Male
7,50.75,400m hurdles,Male
8,8:55.05,3000m steeplechase,Male
9,39.36,4 x 100m relay,Male


In [1961]:
# Converts any time format into seconds

def convert_time(i, string, metric):

    global output
    
    l=['discus', 'throw', 'jump', 'vault', 'shot']
        
    string=string.lower()    
    
    if any(s in string for s in l)==True:
        
        if 'm' in metric:
            
            metric=metric.replace('m', '')
            output=float(str(metric))
            
        elif 'GR' in metric:
            
            metric=metric.replace('GR', '')
            output=float(str(metric))


        else:
    
            output=float(str(metric))
        
    else:
        
        searchstring = ":"
        searchstring2 = "."
        substring=str(metric)
        count = substring.count(searchstring)
        count2 = substring.count(searchstring2)
            
        if count==0:
            output=float(substring)
                        
             
        elif (type(metric)==datetime.time or type(metric)==datetime.datetime):
                                                
            time=str(metric)
            h, m ,s = time.split(':')
            output = float(datetime.timedelta(hours=int(h),minutes=int(m),seconds=float(s)).total_seconds())
            
                                
        elif (count==1 and count2==1):
            
            m,s = metric.split(':')
            output = float(datetime.timedelta(minutes=int(m),seconds=float(s)).total_seconds())
            
            if output==229.90:
                print(metric, m, s, output, 'here')

                     
        elif (count==1 and count2==2):
            
            metric = metric.replace(".", ":", 1)
            
            h,m,s = metric.split(':')            
            output = float(datetime.timedelta(hours=int(h),minutes=int(m),seconds=float(s)).total_seconds())
                
        
        elif (count==2 and count2==0):
            
            h,m,s = metric.split(':')
            output = float(datetime.timedelta(hours=int(h),minutes=int(m),seconds=float(s)).total_seconds())
            
                
    return output

In [1962]:
for i in range(len(benchmarks)):
        
    rowIndex = benchmarks.index[i]

    input_string=benchmarks.iloc[rowIndex,1]
    
    metric=benchmarks.iloc[rowIndex,0]
    
    if metric==None:
        continue
        
    out = convert_time(i, input_string, metric)
     
    benchmarks.loc[rowIndex, 'Metric'] = out

In [1963]:
benchmarks

,BENCHMARK,EVENT,GENDER,Metric
0,10.443,100m,Male,10.443
1,21.02,200m,Male,21.020
2,46.63,400m,Male,46.630
3,1:53.86,800m,Male,113.860
4,14:43.45,5000m,Male,883.450
5,31:25.55,10000m,Male,1885.550
6,13.855,110m hurdles,Male,13.855
7,50.75,400m hurdles,Male,50.750
8,8:55.05,3000m steeplechase,Male,535.050
9,39.36,4 x 100m relay,Male,39.360


In [1964]:
mask = benchmarks['EVENT'].str.contains(r'jump|throw|Pole|put', na=True)

benchmarks.loc[mask, '2pc']=benchmarks['Metric']*0.98
benchmarks.loc[mask, '35pc']=benchmarks['Metric']*0.965
benchmarks.loc[mask, '5pc']=benchmarks['Metric']*0.95

benchmarks.loc[~mask, '2pc']=benchmarks['Metric']*1.02
benchmarks.loc[~mask, '35pc']=benchmarks['Metric']*1.035
benchmarks.loc[~mask, '5pc']=benchmarks['Metric']*1.05


benchmarks.iloc[5, [1]]='10000m run'
benchmarks.iloc[28, [1]]='10000m run'
benchmarks.iloc[26, [1]]='1500m'


In [1965]:
benchmarks

,BENCHMARK,EVENT,GENDER,Metric,2pc,35pc,5pc
0,10.443,100m,Male,10.443,10.65186,10.808505,10.96515
1,21.02,200m,Male,21.020,21.44040,21.755700,22.07100
2,46.63,400m,Male,46.630,47.56260,48.262050,48.96150
3,1:53.86,800m,Male,113.860,116.13720,117.845100,119.55300
4,14:43.45,5000m,Male,883.450,901.11900,914.370750,927.62250
5,31:25.55,10000m run,Male,1885.550,1923.26100,1951.544250,1979.82750
6,13.855,110m hurdles,Male,13.855,14.13210,14.339925,14.54775
7,50.75,400m hurdles,Male,50.750,51.76500,52.526250,53.28750
8,8:55.05,3000m steeplechase,Male,535.050,545.75100,553.776750,561.80250
9,39.36,4 x 100m relay,Male,39.360,40.14720,40.737600,41.32800


In [1966]:
benchmarks

,BENCHMARK,EVENT,GENDER,Metric,2pc,35pc,5pc
0,10.443,100m,Male,10.443,10.65186,10.808505,10.96515
1,21.02,200m,Male,21.020,21.44040,21.755700,22.07100
2,46.63,400m,Male,46.630,47.56260,48.262050,48.96150
3,1:53.86,800m,Male,113.860,116.13720,117.845100,119.55300
4,14:43.45,5000m,Male,883.450,901.11900,914.370750,927.62250
5,31:25.55,10000m run,Male,1885.550,1923.26100,1951.544250,1979.82750
6,13.855,110m hurdles,Male,13.855,14.13210,14.339925,14.54775
7,50.75,400m hurdles,Male,50.750,51.76500,52.526250,53.28750
8,8:55.05,3000m steeplechase,Male,535.050,545.75100,553.776750,561.80250
9,39.36,4 x 100m relay,Male,39.360,40.14720,40.737600,41.32800


In [1967]:
df = athletes.reset_index().merge(benchmarks.reset_index(), on=['EVENT','GENDER'], how='left')


In [1968]:
df

,index_x,NAME,RESULT,AGE,COMPETITION_RANK,EVENT,DOB,COUNTRY,CATEGORY_EVENT,GENDER,COMPETITION,DATE,index_y,BENCHMARK,Metric,2pc,35pc,5pc
0,0,REUBEN SEET HUIJIE,19:04.4,None,11.0,5000m,None,None,Long,Male,NSG,2023,4.0,14:43.45,883.45,901.1190,914.37075,927.6225
1,1,PRANAV SHREEDHAR,18:58.1,None,5.0,5000m,None,None,Long,Male,NSG,2023,4.0,14:43.45,883.45,901.1190,914.37075,927.6225
2,2,JACOB TAN,18:58.2,None,6.0,5000m,None,None,Long,Male,NSG,2023,4.0,14:43.45,883.45,901.1190,914.37075,927.6225
3,3,SAMUEL TOH JUN XIAN,19:03.8,None,8.0,5000m,None,None,Long,Male,NSG,2023,4.0,14:43.45,883.45,901.1190,914.37075,927.6225
4,4,LIEU YEA JAY ERWIN,19:04.1,None,9.0,5000m,None,None,Long,Male,NSG,2023,4.0,14:43.45,883.45,901.1190,914.37075,927.6225
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21486,21486,"{435: 'Peh, Ying Xuan Ulyss', 324: 'Tan, Codee...",1:02.81,None,12,4 x 100m relay,None,None,Relay,Female,10th Club ZOOM Kindred Spirit Series 2023,2023,32.0,44.58,44.58,45.4716,46.14030,46.8090
21487,21487,"{799: 'Yu Xuan, Lua', 802: 'Annabelle Jiamin, ...",58.60,None,4,4 x 100m relay,None,None,Relay,Female,10th Club ZOOM Kindred Spirit Series 2023,2023,32.0,44.58,44.58,45.4716,46.14030,46.8090
21488,21488,"{983: 'Shah, Sharleez Eimann', 985: 'Pugsley, ...",1:09.86,None,18,4 x 100m relay,None,None,Relay,Female,10th Club ZOOM Kindred Spirit Series 2023,2023,32.0,44.58,44.58,45.4716,46.14030,46.8090
21489,21489,"{1309: 'Ong Chun Yi, Avelyn', 1308: 'Verena Ki...",1:03.22,None,14,4 x 100m relay,None,None,Relay,Female,10th Club ZOOM Kindred Spirit Series 2023,2023,32.0,44.58,44.58,45.4716,46.14030,46.8090


In [1969]:
df[df['EVENT']=='race walk']

,index_x,NAME,RESULT,AGE,COMPETITION_RANK,EVENT,DOB,COUNTRY,CATEGORY_EVENT,GENDER,COMPETITION,DATE,index_y,BENCHMARK,Metric,2pc,35pc,5pc
9575,9575,"TAYLOR, CONNOR",9:12.44,14,7,race walk,2010-04-20,None,Walk,Male,SA Allcomers Meet 1,2024,NaN,NaN,NaN,NaN,NaN,NaN
9576,9576,"KHOO, CLARENCE",9:56.29,14,10,race walk,2010-03-03,None,Walk,Male,SA Allcomers Meet 1,2024,NaN,NaN,NaN,NaN,NaN,NaN
9577,9577,"Nai, Lucius",9:17.60,14,8,race walk,2010-02-10,None,Walk,Male,SA Allcomers Meet 1,2024,NaN,NaN,NaN,NaN,NaN,NaN
9578,9578,"Kong, Kendric",8:33.92,14,3,race walk,2010-12-07,None,Walk,Male,SA Allcomers Meet 1,2024,NaN,NaN,NaN,NaN,NaN,NaN
9579,9579,"Zhao, Wesley",8:15.08,14,1,race walk,2010-02-25,None,Walk,Male,SA Allcomers Meet 1,2024,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16824,16824,"Komathi, Jayaraman",21:56.02,61.0,1,race walk,1961-12-12,None,Walk,Female,SMTFA Track & Field Championship 2023,2023,NaN,NaN,NaN,NaN,NaN,NaN
16839,16839,"Foo, Joanna",22:52.09,63.0,2,race walk,1960-01-14,None,Walk,Female,SMTFA Track & Field Championship 2023,2023,NaN,NaN,NaN,NaN,NaN,NaN
16882,16882,"Freddie Low, Han Chung",22:03.84,73.0,1,race walk,1950-02-07,None,Walk,Male,SMTFA Track & Field Championship 2023,2023,NaN,NaN,NaN,NaN,NaN,NaN
16885,16885,"Kahar, Omar",24:20.03,75.0,2,race walk,1948-05-11,None,Walk,Male,SMTFA Track & Field Championship 2023,2023,NaN,NaN,NaN,NaN,NaN,NaN


In [1970]:
df['RESULT'] = athletes['RESULT'].replace(regex=r'–', value=np.NaN)


In [1971]:
for i in range(len(df)):
        
    rowIndex = df.index[i]

    input_string=df.iloc[rowIndex,5]
    
    
    metric=df.iloc[rowIndex,2]
    
    if metric=='—' or metric=='DQ' or metric=='SCR' or metric=='FS' or metric=='DNQ' or metric=='NH':
        continue
    
        
    out = convert_time(i, input_string, metric)
         
    df.loc[rowIndex, 'RESULT_CONV'] = out

In [1972]:
df

,index_x,NAME,RESULT,AGE,COMPETITION_RANK,EVENT,DOB,COUNTRY,CATEGORY_EVENT,GENDER,COMPETITION,DATE,index_y,BENCHMARK,Metric,2pc,35pc,5pc,RESULT_CONV
0,0,REUBEN SEET HUIJIE,19:04.4,None,11.0,5000m,None,None,Long,Male,NSG,2023,4.0,14:43.45,883.45,901.1190,914.37075,927.6225,1144.40
1,1,PRANAV SHREEDHAR,18:58.1,None,5.0,5000m,None,None,Long,Male,NSG,2023,4.0,14:43.45,883.45,901.1190,914.37075,927.6225,1138.10
2,2,JACOB TAN,18:58.2,None,6.0,5000m,None,None,Long,Male,NSG,2023,4.0,14:43.45,883.45,901.1190,914.37075,927.6225,1138.20
3,3,SAMUEL TOH JUN XIAN,19:03.8,None,8.0,5000m,None,None,Long,Male,NSG,2023,4.0,14:43.45,883.45,901.1190,914.37075,927.6225,1143.80
4,4,LIEU YEA JAY ERWIN,19:04.1,None,9.0,5000m,None,None,Long,Male,NSG,2023,4.0,14:43.45,883.45,901.1190,914.37075,927.6225,1144.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21486,21486,"{435: 'Peh, Ying Xuan Ulyss', 324: 'Tan, Codee...",1:02.81,None,12,4 x 100m relay,None,None,Relay,Female,10th Club ZOOM Kindred Spirit Series 2023,2023,32.0,44.58,44.58,45.4716,46.14030,46.8090,62.81
21487,21487,"{799: 'Yu Xuan, Lua', 802: 'Annabelle Jiamin, ...",58.60,None,4,4 x 100m relay,None,None,Relay,Female,10th Club ZOOM Kindred Spirit Series 2023,2023,32.0,44.58,44.58,45.4716,46.14030,46.8090,58.60
21488,21488,"{983: 'Shah, Sharleez Eimann', 985: 'Pugsley, ...",1:09.86,None,18,4 x 100m relay,None,None,Relay,Female,10th Club ZOOM Kindred Spirit Series 2023,2023,32.0,44.58,44.58,45.4716,46.14030,46.8090,69.86
21489,21489,"{1309: 'Ong Chun Yi, Avelyn', 1308: 'Verena Ki...",1:03.22,None,14,4 x 100m relay,None,None,Relay,Female,10th Club ZOOM Kindred Spirit Series 2023,2023,32.0,44.58,44.58,45.4716,46.14030,46.8090,63.22


In [1973]:
df["AGE"].fillna(0, inplace=True)
df['AGE'] = df['AGE'].astype('float')

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_20211/2753303150.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["AGE"].fillna(0, inplace=True)


In [1974]:
#rslt_df = df.loc[(df['RESULT_CONV'] < df['2pc']) & (df['AGE']<40) & (df['EVENT']!='Marathon')]

In [1975]:
#rslt_df = df.loc[(((df['CATEGORY_EVENT']=='Jump')|(df['CATEGORY_EVENT']=='Throw'))&(df['RESULT_CONV'] >= df['5pc']) & (df['AGE']<40) & ((df['EVENT']!='Marathon')|(df['AGE']<60) & (df['EVENT']=='Marathon')))]

In [2001]:
rslt_df = df.loc[(((df['CATEGORY_EVENT']=='Mid')|(df['CATEGORY_EVENT']=='Sprint')|(df['CATEGORY_EVENT']=='Long')|(df['CATEGORY_EVENT']=='Hurdles')|(df['CATEGORY_EVENT']=='Walk')|(df['CATEGORY_EVENT']=='Relay')|(df['CATEGORY_EVENT']=='Marathon')|(df['CATEGORY_EVENT']=='Steeple')|(df['CATEGORY_EVENT']=='Pentathlon')|(df['CATEGORY_EVENT']=='Heptathlon')|(df['CATEGORY_EVENT']=='Triathlon'))&(df['RESULT_CONV'] <= df['5pc']) & (df['AGE']<40) & ((df['EVENT']!='Marathon')|(df['AGE']<60) & (df['EVENT']=='Marathon')))|(((df['CATEGORY_EVENT']=='Jump')|(df['CATEGORY_EVENT']=='Throw'))&(df['RESULT_CONV'] >= df['5pc']) & (df['AGE']<40) & ((df['EVENT']!='Marathon')|(df['AGE']<60) & (df['EVENT']=='Marathon')))]

In [2002]:
rslt_df

,index_x,NAME,RESULT,AGE,COMPETITION_RANK,EVENT,DOB,COUNTRY,CATEGORY_EVENT,GENDER,COMPETITION,DATE,index_y,BENCHMARK,Metric,2pc,35pc,5pc,RESULT_CONV
281,281,"LOH DING RONG, ANSON",50.25,0.0,1.0,Discus throw,None,None,Throw,Male,NSG,2023,18.0,50.02,50.020,49.0196,48.269300,47.51900,50.25
284,284,TAN JUN JIE,48.38,0.0,2.0,Discus throw,None,None,Throw,Male,NSG,2023,18.0,50.02,50.020,49.0196,48.269300,47.51900,48.38
301,301,"CHUA JE-AN, GARRETT",00:14.3,0.0,1.0,110m hurdles,None,None,Hurdles,Male,NSG,2023,6.0,13.855,13.855,14.1321,14.339925,14.54775,14.30
385,385,LEE YU FOONG (LI YUFENG),51.28,0.0,1.0,Discus throw,None,None,Throw,Male,NSG,2023,18.0,50.02,50.020,49.0196,48.269300,47.51900,51.28
718,718,Tan Jun Jie,53.4,0.0,1.0,Discus throw,None,None,Throw,Male,NSG,2024,18.0,50.02,50.020,49.0196,48.269300,47.51900,53.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18583,18583,"Yap, Sean Yee",1.71m,28.0,1,High jump,1995-05-01,None,Jump,Female,10th Club ZOOM Kindred Spirit Series 2023,2023,36.0,1.73,1.730,1.6954,1.669450,1.64350,1.71
18906,18906,"Ryan, Praharsh",22.03,25.0,1,200m,1998-10-18,None,Sprint,Male,10th Club ZOOM Kindred Spirit Series 2023,2023,1.0,21.02,21.020,21.4404,21.755700,22.07100,22.03
18907,18907,"Ryan, Praharsh",22.03,25.0,1,200m,1998-10-18,None,Sprint,Male,10th Club ZOOM Kindred Spirit Series 2023,2023,1.0,21.02,21.020,21.4404,21.755700,22.07100,22.03
18908,18908,"Ryan, Praharsh",22.03,25.0,1,200m,1998-10-18,None,Sprint,Male,10th Club ZOOM Kindred Spirit Series 2023,2023,1.0,21.02,21.020,21.4404,21.755700,22.07100,22.03


In [2003]:
mask = rslt_df['CATEGORY_EVENT'].str.contains(r'Jump|Throw', na=True)
rslt_df.loc[mask, 'Delta2'] = rslt_df['RESULT_CONV']-rslt_df['2pc']
rslt_df.loc[mask, 'Delta35'] = rslt_df['RESULT_CONV']-rslt_df['35pc']
rslt_df.loc[mask, 'Delta5'] = rslt_df['RESULT_CONV']-rslt_df['5pc']

rslt_df.loc[~mask, 'Delta2'] =  rslt_df['2pc'] - rslt_df['RESULT_CONV']
rslt_df.loc[~mask, 'Delta35'] = rslt_df['35pc'] - rslt_df['RESULT_CONV']
rslt_df.loc[~mask, 'Delta5'] = rslt_df['5pc'] - rslt_df['RESULT_CONV']



#rslt_df['Delta2']=rslt_df['2pc']-rslt_df['RESULT_CONV']
#rslt_df['Delta35']=rslt_df['35pc']-rslt_df['RESULT_CONV']
#rslt_df['Delta5']=rslt_df['5pc']-rslt_df['RESULT_CONV']
rslt_df=rslt_df.loc[rslt_df['COMPETITION']!='SEA Games']

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_20211/164990800.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rslt_df.loc[mask, 'Delta2'] = rslt_df['RESULT_CONV']-rslt_df['2pc']
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_20211/164990800.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rslt_df.loc[mask, 'Delta35'] = rslt_df['RESULT_CONV']-rslt_df['35pc']
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_20211/164990800.py:4: SettingWithCopyWarning: 
A value is try

In [2004]:
rslt_df

,index_x,NAME,RESULT,AGE,COMPETITION_RANK,EVENT,DOB,COUNTRY,CATEGORY_EVENT,GENDER,...,index_y,BENCHMARK,Metric,2pc,35pc,5pc,RESULT_CONV,Delta2,Delta35,Delta5
281,281,"LOH DING RONG, ANSON",50.25,0.0,1.0,Discus throw,None,None,Throw,Male,...,18.0,50.02,50.020,49.0196,48.269300,47.51900,50.25,1.2304,1.980700,2.73100
284,284,TAN JUN JIE,48.38,0.0,2.0,Discus throw,None,None,Throw,Male,...,18.0,50.02,50.020,49.0196,48.269300,47.51900,48.38,-0.6396,0.110700,0.86100
301,301,"CHUA JE-AN, GARRETT",00:14.3,0.0,1.0,110m hurdles,None,None,Hurdles,Male,...,6.0,13.855,13.855,14.1321,14.339925,14.54775,14.30,-0.1679,0.039925,0.24775
385,385,LEE YU FOONG (LI YUFENG),51.28,0.0,1.0,Discus throw,None,None,Throw,Male,...,18.0,50.02,50.020,49.0196,48.269300,47.51900,51.28,2.2604,3.010700,3.76100
718,718,Tan Jun Jie,53.4,0.0,1.0,Discus throw,None,None,Throw,Male,...,18.0,50.02,50.020,49.0196,48.269300,47.51900,53.40,4.3804,5.130700,5.88100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18583,18583,"Yap, Sean Yee",1.71m,28.0,1,High jump,1995-05-01,None,Jump,Female,...,36.0,1.73,1.730,1.6954,1.669450,1.64350,1.71,0.0146,0.040550,0.06650
18906,18906,"Ryan, Praharsh",22.03,25.0,1,200m,1998-10-18,None,Sprint,Male,...,1.0,21.02,21.020,21.4404,21.755700,22.07100,22.03,-0.5896,-0.274300,0.04100
18907,18907,"Ryan, Praharsh",22.03,25.0,1,200m,1998-10-18,None,Sprint,Male,...,1.0,21.02,21.020,21.4404,21.755700,22.07100,22.03,-0.5896,-0.274300,0.04100
18908,18908,"Ryan, Praharsh",22.03,25.0,1,200m,1998-10-18,None,Sprint,Male,...,1.0,21.02,21.020,21.4404,21.755700,22.07100,22.03,-0.5896,-0.274300,0.04100


In [2005]:
rslt_df['NAME'] = rslt_df['NAME'].replace(regex=r'PRAHARSH, RYAN', value='S/O SUBASH SOMAN, PRAHARSH RYAN')
rslt_df['NAME'] = rslt_df['NAME'].replace(regex=r'Ryan, Praharsh', value='S/O SUBASH SOMAN, PRAHARSH RYAN')
rslt_df['NAME'] = rslt_df['NAME'].replace(regex=r'TAN, ELIZABETH ANN SHEE R', value='TAN, ELIZABETH-ANN')
rslt_df['NAME'] = rslt_df['NAME'].replace(regex=r'Tan, Elizabeth Ann', value='TAN, ELIZABETH-ANN')

rslt_df['NAME'] = rslt_df['NAME'].replace(regex=r'LOUIS, MARC BRIAN', value='Louis, Marc Brian')
rslt_df['NAME'] = rslt_df['NAME'].replace(regex=r'Louis, Marc', value='Louis, Marc Brian')
rslt_df['NAME'] = rslt_df['NAME'].replace(regex=r'TAN JUN JIE', value='Tan Jun Jie')

rslt_df['NAME'] = rslt_df['NAME'].replace(regex=r'SNG, MICHELLE', value='Sng, Michelle')
rslt_df['NAME'] = rslt_df['NAME'].replace(regex=r'SNG, SUAT LI, MICHELLE', value='Sng, Michelle')

rslt_df['NAME'] = rslt_df['NAME'].replace(regex=r'MUN, IVAN', value='Mun, Ivan')
rslt_df['NAME'] = rslt_df['NAME'].replace(regex=r'LOW, JUN YU', value='Low, Jun Yu')

rslt_df['NAME'] = rslt_df['NAME'].replace(regex=r'ANG, CHEN XIANG', value='Ang, Chen Xiang')
rslt_df['NAME'] = rslt_df['NAME'].replace(regex=r'LIM, OLIVER', value='Lim, Oliver')



/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_20211/180801585.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rslt_df['NAME'] = rslt_df['NAME'].replace(regex=r'PRAHARSH, RYAN', value='S/O SUBASH SOMAN, PRAHARSH RYAN')
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_20211/180801585.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rslt_df['NAME'] = rslt_df['NAME'].replace(regex=r'Ryan, Praharsh', value='S/O SUBASH SOMAN, PRAHARSH RYAN')
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t400

In [2006]:
rslt_df['PERF_SCALAR']=rslt_df['Delta5']/rslt_df['Metric']*100

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_20211/1814660016.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rslt_df['PERF_SCALAR']=rslt_df['Delta5']/rslt_df['Metric']*100


In [2007]:
rslt_df.to_csv('checkpoint.csv', sep=',', encoding='utf-8-sig', index=False)


In [2008]:
#rslt_df['Rank'] = (rslt_df.sort_values(by=['EVENT', 'GENDER', 'Delta35'], ascending=[False, False, True])['Delta35']
#                .rank(method='first', ascending=False)
#             )


In [2009]:
spex_list=pd.read_csv('/Users/veesheenyuen/Desktop/DataScience/SAA/SPEX_CARDED_LIST.csv', encoding='latin-1')


In [2010]:
spex_list

,V1,V2,V3,V4,Level
0,Goh Chui Ling,"Goh, Chui Ling",NaN,NaN,E3
1,Michelle Sng,"Sng, Michelle",NaN,NaN,E3
2,Quek Jun Jie Calvin,"Quek, Jun Jie Calvin",NaN,NaN,E3
3,"SOH RUI YONG, GUILLAUME","Soh Rui Yong, Guillaume",NaN,NaN,E3
4,Aaron Justin tan wen jie,"Tan Wen Jie, Aaron Justin",NaN,NaN,E3P
5,DANIEL LEOW SOON YEE,"Yee, Daniel Leow Soon",NaN,NaN,E3P
6,Joshua Chua,"Chua, Joshua",NaN,NaN,E3P
7,NG ZHI RONG RYAN RAPHAEL,"Ng Zhi Rong, Ryan Raphael","Raphael, Ryan",NaN,E3P
8,Wenli Rachel,"Wenli, Rachel",NaN,NaN,E3P
9,Wong YaoHan Melvin,"Wong YaoHan, Melvin",NaN,NaN,E3P


In [2011]:
spex1 = spex_list['V1'].dropna().tolist()
spex2 = spex_list['V2'].dropna().tolist()
spex3 = spex_list['V3'].dropna().tolist()
spex4 = spex_list['V4'].dropna().tolist()


In [2012]:
spex_athletes=spex1+spex2+spex3+spex4

In [2013]:
spex_athletes

['Goh Chui Ling',
 'Michelle Sng',
 'Quek Jun Jie Calvin',
 'SOH RUI YONG, GUILLAUME',
 'Aaron Justin tan wen jie',
 'DANIEL LEOW SOON YEE',
 'Joshua Chua',
 'NG ZHI RONG RYAN RAPHAEL',
 'Wenli Rachel',
 'Wong YaoHan Melvin',
 'XANDER HO ANN HENG',
 'Veronica Shanti Pereira',
 'Ang Chen Xiang',
 'KAMPTON KAM',
 'MARC BRIAN LOUIS',
 'Mark Lee Ren',
 'Reuben Rainer Lee Siong En',
 'ELIZABETH-ANN TAN SHEE RU',
 'Thiruben Thana Rajan',
 'Bhavna Gopikrishna',
 'Chloe Chee En-Ya',
 'Conrad Kangli Emery',
 'Harry Irfan Curran',
 'Huang weijun',
 'Jayden tan',
 'Koh Shun Yi Audrey',
 'LAAVINIA D/O JAIGANTH',
 'Lim Yee Chern Clara',
 'LOH DING RONG ANSON',
 'ONG YING TAT',
 'SONG EN XU REAGAN',
 'Subaraghav hari',
 'Teh Ying Shan',
 'YAN TEO',
 'ZHONG CHUHAN',
 'ESTHER TAY SHEE WEI',
 'Faith Ford',
 'Garrett Chua Je-An',
 'Lucas Fun',
 'Goh, Chui Ling',
 'Sng, Michelle',
 'Quek, Jun Jie Calvin',
 'Soh Rui Yong, Guillaume',
 'Tan Wen Jie, Aaron Justin',
 'Yee, Daniel Leow Soon',
 'Chua, Joshua',

In [2014]:
spex_athletes_casefold=[s.casefold() for s in spex_athletes]

In [2015]:
spex_athletes_casefold

['goh chui ling',
 'michelle sng',
 'quek jun jie calvin',
 'soh rui yong, guillaume',
 'aaron justin tan wen jie',
 'daniel leow soon yee',
 'joshua chua',
 'ng zhi rong ryan raphael',
 'wenli rachel',
 'wong yaohan melvin',
 'xander ho ann heng',
 'veronica shanti pereira',
 'ang chen xiang',
 'kampton kam',
 'marc brian louis',
 'mark lee ren',
 'reuben rainer lee siong en',
 'elizabeth-ann tan shee ru',
 'thiruben thana rajan',
 'bhavna gopikrishna',
 'chloe chee en-ya',
 'conrad kangli emery',
 'harry irfan curran',
 'huang weijun',
 'jayden tan',
 'koh shun yi audrey',
 'laavinia d/o jaiganth',
 'lim yee chern clara',
 'loh ding rong anson',
 'ong ying tat',
 'song en xu reagan',
 'subaraghav hari',
 'teh ying shan',
 'yan teo',
 'zhong chuhan',
 'esther tay shee wei',
 'faith ford',
 'garrett chua je-an',
 'lucas fun',
 'goh, chui ling',
 'sng, michelle',
 'quek, jun jie calvin',
 'soh rui yong, guillaume',
 'tan wen jie, aaron justin',
 'yee, daniel leow soon',
 'chua, joshua',

In [2016]:
top_performers=rslt_df.sort_values(['NAME','PERF_SCALAR'],ascending=False).groupby('NAME').head(1) # Choose top performing event per NAME


In [2017]:
top_performers

,index_x,NAME,RESULT,AGE,COMPETITION_RANK,EVENT,DOB,COUNTRY,CATEGORY_EVENT,GENDER,...,BENCHMARK,Metric,2pc,35pc,5pc,RESULT_CONV,Delta2,Delta35,Delta5,PERF_SCALAR
14935,14935,"{80: 'Jirapat, KHANONTA\xa0', 79: 'Manatsada, ...",46.06,0.0,3,4 x 100m relay,None,None,Relay,Female,...,44.58,44.580,45.47160,46.140300,46.80900,46.06,-0.58840,0.080300,0.74900,1.680126
14918,14918,"{71: 'KANHAAUDOM, Methawi', 74: 'Joseph Obioma...",3:14.84,0.0,2,4 x 400m relay,None,None,Relay,Male,...,3:08.82,188.820,192.59640,195.428700,198.26100,194.84,-2.24360,0.588700,3.42100,1.811778
9522,9522,"{36: 'QUEK, CALVIN', 35: 'CHUA, JOSHUA', 37: '...",40.56,0.0,1,4 x 100m relay,None,None,Relay,Male,...,39.36,39.360,40.14720,40.737600,41.32800,40.56,-0.41280,0.177600,0.76800,1.951220
14908,14908,"{26: 'ROSLAN, ABDUL WAFIY', 29: 'ROSELI, RUSLE...",3:14.46,0.0,1,4 x 400m relay,None,None,Relay,Male,...,3:08.82,188.820,192.59640,195.428700,198.26100,194.46,-1.86360,0.968700,3.80100,2.013028
14941,14941,"{260: 'ZULKIFLI, ZAIDATUL HUSNIAH', 261: 'ALIA...",45.70,0.0,1,4 x 100m relay,None,None,Relay,Female,...,44.58,44.580,45.47160,46.140300,46.80900,45.70,-0.22840,0.440300,1.10900,2.487663
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15017,15017,"Amjaram, Vinsensia Awutet",5.75m,26.0,1,Long jump,1998-06-04 00:00:00.000,None,Jump,Female,...,6.02,6.020,5.89960,5.809300,5.71900,5.75,-0.14960,-0.059300,0.03100,0.514950
14630,14630,"ANURA, ANDRE",14.93m,24.0,2,Triple jump,1999-06-12,None,Jump,Male,...,15.7,15.700,15.38600,15.150500,14.91500,14.93,-0.45600,-0.220500,0.01500,0.095541
15101,15101,"ANG, JAMIE EL-REDHA",10.80,20.0,5,100m,2004-01-06 00:00:00.000,None,Sprint,Male,...,10.443,10.443,10.65186,10.808505,10.96515,10.80,-0.14814,0.008505,0.16515,1.581442
14873,14873,"ALING, NUR AISHAH ROFINA",12.09,24.0,4,100m,1999-01-31,None,Sprint,Female,...,11.75,11.750,11.98500,12.161250,12.33750,12.09,-0.10500,0.071250,0.24750,2.106383


In [2018]:
top_performers.to_csv('top_checkpoint.csv', sep=',', encoding='utf-8-sig', index=False)


In [2019]:
spexed_list = top_performers.loc[~rslt_df['NAME'].str.casefold().isin(spex_athletes_casefold)]  # ~ means NOT IN. DROP spex carded athletes

In [1995]:
#spexed_list=top_performers

In [2020]:
spexed_list.sort_values(['EVENT', 'GENDER', 'PERF_SCALAR'], ascending=[True, True, False], inplace=True)
spexed_list['overall_rank'] = 1
spexed_list['overall_rank'] = spexed_list.groupby(['EVENT', 'GENDER'])['overall_rank'].cumsum()

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_20211/1563010327.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spexed_list.sort_values(['EVENT', 'GENDER', 'PERF_SCALAR'], ascending=[True, True, False], inplace=True)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_20211/1563010327.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spexed_list['overall_rank'] = 1
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_20211/1563010327.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Tr

In [2021]:
#spexed_list=spexed_list[(((spexed_list['EVENT']=='400m')&(spexed_list['overall_rank']<7)))|(((spexed_list['EVENT']=='100m')&(spexed_list['overall_rank']<7)))]

In [2022]:
spexed_list=spexed_list[(((spexed_list['EVENT']=='400m')|(spexed_list['EVENT']=='100m'))&(spexed_list['overall_rank']<7))|(~((spexed_list['EVENT']=='400m')|(spexed_list['EVENT']=='100m'))&(spexed_list['overall_rank']<4))]


In [2023]:
spexed_list

,index_x,NAME,RESULT,AGE,COMPETITION_RANK,EVENT,DOB,COUNTRY,CATEGORY_EVENT,GENDER,...,Metric,2pc,35pc,5pc,RESULT_CONV,Delta2,Delta35,Delta5,PERF_SCALAR,overall_rank
15268,15268,"Lim, Jon",32:54.40,0.0,1,10000m run,None,None,Long,Male,...,1885.55,1923.261,1951.54425,1979.8275,1974.40,-51.139,-22.85575,5.4275,0.287847,1
14863,14863,"HU, Chia-Chen",11.57,26.0,2,100m,1997-03-28,None,Sprint,Female,...,11.75,11.985,12.16125,12.3375,11.57,0.415,0.59125,0.7675,6.531915,1
14873,14873,"ALING, NUR AISHAH ROFINA",12.09,24.0,4,100m,1999-01-31,None,Sprint,Female,...,11.75,11.985,12.16125,12.3375,12.09,-0.105,0.07125,0.2475,2.106383,2
14528,14528,"Sagi, Mayako",12.16,21.0,1,100m,2003-01-03,None,Sprint,Female,...,11.75,11.985,12.16125,12.3375,12.16,-0.175,0.00125,0.1775,1.510638,3
14870,14870,"Sukanda, PETRAKSA",12.20,20.0,5,100m,2003-08-15,None,Sprint,Female,...,11.75,11.985,12.16125,12.3375,12.20,-0.215,-0.03875,0.1375,1.170213,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14589,14589,"CHEN, Yi Chu",16.87m,21.0,1,Shot put,2002-10-31,None,Throw,Male,...,17.30,16.954,16.69450,16.4350,16.87,-0.084,0.17550,0.4350,2.514451,1
14588,14588,"RIGAN, JONAH CHANG",16.50m,20.0,2,Shot put,2003-03-25,None,Throw,Male,...,17.30,16.954,16.69450,16.4350,16.50,-0.454,-0.19450,0.0650,0.375723,2
14966,14966,"LEE, MINJE",15.20m,23.0,1,Triple jump,2001-04-11 00:00:00.000,None,Jump,Male,...,15.70,15.386,15.15050,14.9150,15.20,-0.186,0.04950,0.2850,1.815287,1
14631,14631,"Ting Li King, Brandon",15.03m,21.0,1,Triple jump,2002-01-09,None,Jump,Male,...,15.70,15.386,15.15050,14.9150,15.03,-0.356,-0.12050,0.1150,0.732484,2


In [2024]:
spexed_list.to_csv('ranked_all_events.csv', sep=',', encoding='utf-8-sig', index=False)


In [ ]:
# 1. If top athlete>30 (and the only one >30), there are already 6 in list, and next one is already <30 then it means do nothing? 
# 2. If top athlete>30 and is the sole pick.  Next athlete is far beyond 5% band.  Do we add?
# 3. If top athlete>30 and there are less than 6 within 5% band of SEAG benchmark. Do we add that one more who is beyond 5%?
# 4. At least one per gender.  The only pick is far beyond 5%.  Do we add?